In [52]:
# after we install all the packages, import all of them for the use in today's lecture!
# database access
import pandas_datareader as web
import quandl as quandl
import wrds as wrds
# storage and operations
import pandas as pd
import numpy as np
import datetime
from pathlib import Path
import seaborn as sns

# stats
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from scipy import stats
from scipy.stats.mstats import winsorize
import os
from tqdm import tqdm
from zipfile import ZipFile
import seaborn
import matplotlib.pyplot as plt
import linearmodels as lm
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

#portfolio optimization:
# libraries we might use for testing
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
import linearmodels as lm
from linearmodels.panel import compare  
import time
import quantstats as qs

from datetime import datetime

path_factors = Path(r'C:\Users\mauri\FSoF\Frankfurt School of Finance - Master Thesis\Master Thesis - Data\Factors')

path_benchmark_factors = Path(r'C:\Users\mauri\FSoF\Frankfurt School of Finance - Master Thesis\Master Thesis - Data\Factors\Benchmark Factors (B.Kelly)')
benchmark_factor_data_csv = path_factors / 'benchmark_factor_data.csv'
benchmark_factor_data_bkelly_csv = path_factors /' benchmark_factor_data_bkelly.csv'

In [38]:
benchmark_factors = pd.DataFrame()

momentum = pd.read_csv(path_benchmark_factors/'[usa]_[momentum]_[monthly]_[vw].csv')
momentum = momentum.rename(columns={'ret':'momentum'})
momentum = momentum[['date','momentum']]

investment = pd.read_csv(path_benchmark_factors/'[usa]_[investment]_[monthly]_[vw].csv')
investment = investment.rename(columns={'ret':'investment'})
investment = investment['investment']


profitability = pd.read_csv(path_benchmark_factors/'[usa]_[profitability]_[monthly]_[vw].csv')
profitability = profitability.rename(columns={'ret':'profitability'})
profitability = profitability['profitability']

quality = pd.read_csv(path_benchmark_factors/'[usa]_[quality]_[monthly]_[vw].csv')
quality = quality.rename(columns={'ret':'quality'})
quality = quality['quality']

short_term_reversal = pd.read_csv(path_benchmark_factors/'[usa]_[short_term_reversal]_[monthly]_[vw].csv')
short_term_reversal = short_term_reversal.rename(columns={'ret':'str'})
short_term_reversal = short_term_reversal['str']

size = pd.read_csv(path_benchmark_factors/'[usa]_[size]_[monthly]_[vw].csv')
size = size.rename(columns={'ret':'size'})
size = size['size']

value = pd.read_csv(path_benchmark_factors/'[usa]_[value]_[monthly]_[vw].csv')
value = value.rename(columns={'ret':'value'})
value = value['value']

low_risk = pd.read_csv(path_benchmark_factors/'[usa]_[low_risk]_[monthly]_[vw].csv')
low_risk = low_risk.rename(columns={'ret':'low_risk'})
low_risk = low_risk['low_risk']


benchmark_factors = pd.concat([benchmark_factors,momentum],axis=1)
benchmark_factors = pd.concat([benchmark_factors,investment],axis=1)
benchmark_factors = pd.concat([benchmark_factors,profitability],axis=1)
benchmark_factors = pd.concat([benchmark_factors,quality],axis=1)
benchmark_factors = pd.concat([benchmark_factors,short_term_reversal],axis=1)
benchmark_factors = pd.concat([benchmark_factors,size],axis=1)
benchmark_factors = pd.concat([benchmark_factors,value],axis=1)
benchmark_factors = pd.concat([benchmark_factors,low_risk],axis=1)

benchmark_factors['date'] = pd.to_datetime(benchmark_factors['date'])
benchmark_factors['date'] =benchmark_factors['date'].dt.strftime('%Y-%m')
# benchmark_factors = benchmark_factors.loc[benchmark_factors.date=='1963-08']
benchmark_factors = benchmark_factors.sort_index(1)
benchmark_factors = benchmark_factors.loc[(benchmark_factors.date>=('1977-07'))&(benchmark_factors.date<=('2020-12'))]
benchmark_factors

C:\Users\mauri\AppData\Local\Temp\ipykernel_1928\3431566630.py:49: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  benchmark_factors = benchmark_factors.sort_index(1)


,date,investment,low_risk,momentum,profitability,quality,size,str,value
615,1977-07,-0.005338,0.008454,0.002983,0.001553,0.032875,0.009605,-0.008720,0.013068
616,1977-08,0.001718,-0.030438,-0.015859,0.012426,0.008731,0.008529,0.000533,-0.021223
617,1977-09,0.028758,0.009018,0.015039,0.002766,0.003922,0.011287,0.002593,-0.011110
618,1977-10,0.024382,-0.007422,0.001249,-0.001897,0.014960,0.014257,-0.012740,-0.010747
619,1977-11,0.003730,0.004260,0.011796,-0.010240,0.005450,0.008628,0.004856,-0.009513
...,...,...,...,...,...,...,...,...,...
1132,2020-08,NaN,-0.046453,0.041055,0.013158,-0.023391,0.000750,0.003614,-0.004821
1133,2020-09,NaN,0.012753,-0.003680,-0.013201,0.041765,0.020071,-0.000728,0.046081
1134,2020-10,NaN,-0.043156,-0.023437,-0.014593,0.044531,-0.045658,-0.028804,-0.065966
1135,2020-11,NaN,0.007035,-0.074427,-0.098790,0.007989,-0.024356,0.016040,0.010607


In [48]:
ff_factors = pd.DataFrame(pd.read_csv(path_factors/'Fama-French/FF Factor Monthly/F-F_Research_Data_5_Factors_2x3_CSV.zip',header=2,nrows=704))
ff_mom = pd.DataFrame(pd.read_csv(path_factors/'Fama-French/FF Factor Monthly/F-F_Momentum_Factor_CSV.zip',header=11,nrows= 1144))


ff_factors = ff_factors.rename(columns={'Unnamed: 0' : 'date'})
ff_mom = ff_mom.rename(columns={'Unnamed: 0':'date'})

ff_factors['date'] = pd.to_datetime(ff_factors['date'],format='%Y%m')
ff_mom['date'] = pd.to_datetime(ff_mom['date'],format='%Y%m')

ff_factors = ff_factors.merge(ff_mom,how='left', on='date')
ff_factors.columns = [z.lower().strip() for z in ff_factors.columns]

ff_factors['date'] =ff_factors['date'].dt.strftime('%Y-%m')
ff_factors = ff_factors.set_index('date')
ff_factors = ff_factors/100
ff_factors = ff_factors[['mkt-rf','smb','hml', 'rmw','cma', 'mom', 'rf']] 
ff_factors

,mkt-rf,smb,hml,rmw,cma,mom,rf
date,,,,,,,
1963-07,-0.0039,-0.0044,-0.0089,0.0068,-0.0123,0.0097,0.0027
1963-08,0.0507,-0.0075,0.0168,0.0036,-0.0034,0.0101,0.0025
1963-09,-0.0157,-0.0055,0.0008,-0.0071,0.0029,0.0019,0.0027
1963-10,0.0253,-0.0137,-0.0014,0.0280,-0.0202,0.0312,0.0029
1963-11,-0.0085,-0.0089,0.0181,-0.0051,0.0231,-0.0074,0.0027
...,...,...,...,...,...,...,...
2021-10,0.0665,-0.0271,-0.0048,0.0174,-0.0151,0.0319,0.0000
2021-11,-0.0155,-0.0173,-0.0042,0.0738,0.0163,0.0089,0.0000
2021-12,0.0310,-0.0069,0.0322,0.0475,0.0436,-0.0260,0.0001


In [51]:
################################################# saving  ##################################################
ff_factors.to_csv(benchmark_factor_data_csv) #, compression='zip', encoding='unicode_escape-8'
############################################################################################################

In [53]:
################################################# saving  ##################################################
# benchmark_factors.to_csv(benchmark_factor_data_bkelly_csv) #, compression='zip', encoding='unicode_escape-8'
############################################################################################################